In [1]:
%pip install gpt-2-simple
%pip install tensorflow==1.14
%pip install tensorflow-gpu==1.14.0

  Created wheel for gpt-2-simple: filename=gpt_2_simple-0.7.1-cp36-none-any.whl size=23580 sha256=b0e269321e4ab99681e403d1d31d514eab8c1dafc14b4cd02a43348fc5f306e7
  Stored in directory: /root/.cache/pip/wheels/0c/f8/23/b53ce437504597edff76bf9c3b8de08ad716f74f6c6baaa91a
Successfully built gpt-2-simple
     |████████████████████████████████| 109.2MB 93kB/s 
     |████████████████████████████████| 3.2MB 50.3MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 491kB 49.0MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 377.0MB 48

In [3]:
import os
import sys
import requests
from tqdm import tqdm

def download_model(model="124M"):
  subdir = os.path.join('models', model)
  if not os.path.exists(subdir):
      os.makedirs(subdir)

  for filename in ['checkpoint','encoder.json','hparams.json','model.ckpt.data-00000-of-00001', 'model.ckpt.index', 'model.ckpt.meta', 'vocab.bpe']:
      r = requests.get("https://openaipublic.blob.core.windows.net/gpt-2/" + subdir + "/" + filename, stream=True)
      with open(os.path.join(subdir, filename), 'wb') as f:
          file_size = int(r.headers["content-length"])
          chunk_size = 1000
          with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
              for chunk in r.iter_content(chunk_size=chunk_size):
                  f.write(chunk)
                  pbar.update(chunk_size)

In [26]:
import gpt_2_simple as gpt2
import os
import requests

model_name = "124M"
download_model(model=model_name)

file_name = "sample_letters.txt" # provide file for training

sess = gpt2.start_tf_sess()
gpt2.finetune(
    sess,
    file_name,
    model_name=model_name,
    steps=1000,
    print_every=25,
    sample_every=200,
    save_every=500
)


Fetching checkpoint: 1.00kit [00:00, 407kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:01, 651kit/s]                                                    
Fetching hparams.json: 1.00kit [00:00, 499kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:53, 9.36Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 1.59Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:01, 434kit/s]                                                  
Fetching vocab.bpe: 457kit [00:01, 428kit/s]                                                        


ValueError: ignored

In [ ]:
generated = gpt2.generate(
    sess,
    length=250,
    temperature=0.7,
    prefix="With you....",
    return_as_list=True,
    run_name='run1'
)

print(generated)

In [ ]:
import json

prefixes = None
with open("prefixes", "r") as prefixes_file:
  prefixes = json.loads(prefixes_file.read())

assert prefixes is not None

generated = []
for prefix in prefixes:
  g = gpt2.generate(
      sess,
      length=250,
      temperature=0.7,
      prefix=prefix,
      return_as_list=True,
      run_name='run1'
  )
  generated.append({prefix: g[0]})
  print('generated', generated[-1])

print('generation finished.')

In [ ]:
with open("generated", "w") as f:
  json.dump(generated, f)